# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,28.67,91,31,10.29,GS,1691550844
1,1,kaa-khem,51.6833,94.7333,66.83,78,100,0.87,RU,1691550844
2,2,weno,7.4515,151.8468,85.95,79,100,8.05,FM,1691550845
3,3,puerto natales,-51.7236,-72.4875,36.07,95,100,4.27,CL,1691550637
4,4,bhawanigarh,30.2667,76.0350,86.11,68,100,7.02,IN,1691550845


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",  
    frame_width=700,
    frame_height=500,
    hover_cols=["City", "Humidity"],  
)

# Display the map
map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria
filtered_cities_df = city_data_df[
    (city_data_df['Max Temp'] > 70) &  # Criteria: Max Temperature greater than 70
    (city_data_df['Max Temp'] < 80)   # Criteria: Max Temperature less than 80
]

# Drop any rows with null values
filtered_cities_df.dropna(inplace=True)

# Display sample data
filtered_cities_df.head()

/var/folders/k_/y8x1yx1949sf2yq9wp0mkzvh0000gp/T/ipykernel_87320/293193415.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_cities_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,avarua,-21.2078,-159.7750,75.25,60,20,10.36,CK,1691550728
12,12,corinth,37.9407,22.9573,79.90,72,1,5.14,GR,1691550846
13,13,baker city,44.7749,-117.8344,73.58,62,0,6.91,US,1691550846
20,20,adamstown,-25.0660,-130.1015,72.01,85,95,34.52,PN,1691550650
22,22,port mathurin,-19.6833,63.4167,72.09,61,95,18.34,MU,1691550658


In [5]:
import requests
from requests.structures import CaseInsensitiveDict

url = "https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=rect%3A10.716463143326969%2C48.755151258420966%2C10.835314015356737%2C48.680903341613316&limit=20&apiKey=5edc1bba7b49463f99923787dc91468c"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"

resp = requests.get(url, headers=headers)

print(resp.status_code)

200


In [6]:
print(geoapify_key)

cf7c0762d5e34616a19b1495fd9d1b5a


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df.copy()[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
11,avarua,CK,-21.2078,-159.7750,60,
12,corinth,GR,37.9407,22.9573,72,
13,baker city,US,44.7749,-117.8344,62,
20,adamstown,PN,-25.0660,-130.1015,85,
22,port mathurin,MU,-19.6833,63.4167,61,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "type": "accommodation.hotel",
    "filter": filter,
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{lng},{radius}"
    #params["bias"] = f"proximity:{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    print(response)
    # Convert the API response to JSON format
    name_address = response.json()
    print(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
<Response [200]>
{'type': 'FeatureCollection', 'features': []}
avarua - nearest hotel: No hotel found
<Response [200]>
{'type': 'FeatureCollection', 'features': []}
corinth - nearest hotel: No hotel found
<Response [200]>
{'type': 'FeatureCollection', 'features': []}
baker city - nearest hotel: No hotel found
<Response [200]>
{'type': 'FeatureCollection', 'features': []}
adamstown - nearest hotel: No hotel found
<Response [200]>
{'type': 'FeatureCollection', 'features': []}
port mathurin - nearest hotel: No hotel found
<Response [200]>
{'type': 'FeatureCollection', 'features': []}
amursk - nearest hotel: No hotel found
<Response [200]>
{'type': 'FeatureCollection', 'features': []}
taiohae - nearest hotel: No hotel found
<Response [200]>
{'type': 'FeatureCollection', 'features': []}
tamandare - nearest hotel: No hotel found
<Response [200]>
{'type': 'FeatureCollection', 'features': []}
acarau - nearest hotel: No hotel found
<Response [200]>
{'type': 'FeatureCollect

,City,Country,Lat,Lng,Humidity,Hotel Name
11,avarua,CK,-21.2078,-159.7750,60,No hotel found
12,corinth,GR,37.9407,22.9573,72,No hotel found
13,baker city,US,44.7749,-117.8344,62,No hotel found
20,adamstown,PN,-25.0660,-130.1015,85,No hotel found
22,port mathurin,MU,-19.6833,63.4167,61,No hotel found
...,...,...,...,...,...,...
578,harstad,NO,68.7990,16.5293,83,No hotel found
579,zhezqazghan,KZ,47.7833,67.7667,68,No hotel found
580,kirensk,RU,57.7853,108.1119,52,No hotel found
582,taber,CA,49.8167,-112.1518,55,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
map = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    tiles="OSM",  
    frame_width=700,
    frame_height=500,
    hover_cols=["Hotel Name", "Country"],  
)

map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)